In [ ]:
import sqlite3
from sqlite3 import Error

import pandas as pd
import requests
import urllib.parse
import re

In [ ]:
def read_data_from_DB():
    try:
        path_to_DB = r"../Data/house.db"
        conn = sqlite3.connect(path_to_DB)
        df = pd.read_sql_query("SELECT * from house_features", conn)
    except Error as e:
        print(e)
        df = 0
    return df
df = read_data_from_DB()

In [ ]:
df.head(5)


In [ ]:
def get_coords(address):
    clean_address = re.match(r".+?(\d+)",address)
    if clean_address:
        url = "https://nominatim.openstreetmap.org/?addressdetails=1&q=" + clean_address.group()  +"&format=json&limit=1"
        response = requests.get(url).json()
        if response:
            res = response[0]["lat"],response[0]["lon"]
            print(res)
            return res
        else:
            return -1,-1 # Not found
    return -1,-1


df["COORDS"] = df["ADDRESS"].map(lambda x: get_coords(x))
df.head()


## TODO:

- Get coords
- Compute distance from Helsinki city center
- EDA on features + cleaning
- Regerssion on prices
    Naive + XGboost + ...